<div style="display: flex; background-color: RGB(255,114,0);" >
<h1 style="margin: auto; padding: 30px; ">Brief 9 -  </h1>
</div>

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Partie 3 - Feature engineering </h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 - Importation des librairies</h3>
</div>

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
from clean.Cleaning import data_check
from clean.Cleaning import record
from clean.Cleaning import missing_values
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import pickle
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

<div style="background-color: RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color:#fff; ">2.2 - Importation et préparation des fichiers</h3>
</div>

In [2]:
data = pd.read_csv(r'../data/CURATED/df_clean.csv', encoding="UTF8", sep=',')
df = data.copy()

In [3]:
a=df[['Id','Type local']].pivot_table(index='Id', columns='Type local', aggfunc=len, fill_value=0).reset_index()

In [4]:
b=df.groupby("Id").agg(
    Valeur_fonciere = pd.NamedAgg(column="Valeur fonciere", aggfunc="max"),
    Nombre_pieces_principales = pd.NamedAgg(column="Nombre pieces principales", aggfunc="sum"),
    tot_lot = pd.NamedAgg(column="Type local", aggfunc="count"),
    typ_lot = pd.NamedAgg(column="Type local", aggfunc="nunique"),
    Surface_reelle_bati = pd.NamedAgg(column="Surface reelle bati", aggfunc="sum")
    ).reset_index(level=0)

In [36]:
c=a.merge(b, how='left', left_on = 'Id', right_on='Id' )

In [6]:
local=pd.DataFrame(df.groupby(['Id'])['Type local'].agg(lambda x: x.value_counts().index[0])).reset_index()

In [7]:
surf = pd.DataFrame(df.groupby(['Id'])['Surface terrain'].agg(lambda x: x.max() if x.nunique()==1 else x.unique().sum())).reset_index()

In [37]:
c['Surface_terrain']=local['Surface terrain']
c['local']=good['Type local']

In [38]:
# plusieurs terrain
c.loc[(c[(c['typ_lot']>1) & (c['Terrain']>1) & (c['Local industriel. commercial ou assimilé']>c['Appartement'])& (c['Local industriel. commercial ou assimilé'] > c['Maison']) & (c['local']=='Terrain')]).index, 'local'] = 'Local industriel. commercial ou assimilé'
c.loc[(c[(c['typ_lot']>1) & (c['Terrain']>1) & (c['Maison']>c['Appartement']) & (c['Maison']>c['Local industriel. commercial ou assimilé']) & (c['local']=='Terrain')]).index, 'local'] = 'Maison'
c.loc[(c[(c['typ_lot']>1) & (c['Terrain']>1) & (c['Appartement']> c['Maison']) & (c['Appartement']>c['Local industriel. commercial ou assimilé']) & (c['local']=='Terrain')]).index, 'local'] = 'Appartement'

In [39]:
# dependance + appartement = appartement
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']!=0) & (c['Local industriel. commercial ou assimilé']==0) 
  & (c['Maison']==0) & (c['Terrain']==0) & (c['local']=='Dépendance')]).index, 'local'] = 'Appartement'

In [40]:
# dependance + Maison = Maison
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']==0) & (c['Local industriel. commercial ou assimilé']==0) 
  & (c['Maison']!=0) & (c['Terrain']==0) & (c['local']=='Dépendance')]).index, 'local'] = 'Maison'

In [41]:
# dependance + Local industriel = Local industriel
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']==0) & (c['Local industriel. commercial ou assimilé']!=0) 
  & (c['Maison']==0) & (c['Terrain']==0) & (c['local']=='Dépendance')]).index, 'local'] = 'Local industriel. commercial ou assimilé'

In [42]:
# dependance + Terrain = Terrain
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']==0) & (c['Local industriel. commercial ou assimilé']==0) 
  & (c['Maison']==0) & (c['Terrain']!=0) & (c['local']=='Dépendance')]).index, 'local'] = 'Terrain'

In [43]:
# dependance + Maison + appartement = Maison
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']!=0) & (c['Local industriel. commercial ou assimilé']==0) 
  & (c['Maison']!=0) & (c['Terrain']==0) & (c['local']=='Dépendance')]).index, 'local'] = 'Maison'

In [44]:
# dependance + Maison + local = Maison
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']==0) & (c['Local industriel. commercial ou assimilé']!=0) 
  & (c['Maison']!=0) & (c['Terrain']==0) & (c['local']=='Dépendance')]).index, 'local'] = 'Maison'

In [45]:
# dependance + local + appartement = appartement
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']!=0) & (c['Local industriel. commercial ou assimilé']!=0) 
  & (c['Maison']==0) & (c['Terrain']==0) & (c['local']=='Dépendance') & (c['Appartement']>=c['Local industriel. commercial ou assimilé'])]).index, 'local'] = 'Appartement'

In [46]:
# dependance + local + appartement = local
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']!=0) & (c['Local industriel. commercial ou assimilé']!=0) 
  & (c['Maison']==0) & (c['Terrain']==0) & (c['local']=='Dépendance') & (c['Appartement']<c['Local industriel. commercial ou assimilé'])]).index, 'local'] = 'Local industriel. commercial ou assimilé'

In [47]:
#  Maison + local = Maison
c.loc[(c[(c['typ_lot']>1) & (c['Maison']==c['Local industriel. commercial ou assimilé']) & (c['local']=='Local industriel. commercial ou assimilé')]).index, 'local'] = 'Maison'

In [48]:
# plusieurs appartements 
c.loc[(c[(c['typ_lot']==1) & (c['Appartement']>1)]).index, 'local'] = 'Batiment'
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']>1) & (c['Maison']==0) & (c['local']=='Appartement')]).index, 'local'] = 'Batiment'
c.loc[(c[(c['typ_lot']>1) & (c['Appartement']>1) & (c['Maison']!=0) & (c['local']=='Appartement')]).index, 'local'] = 'mix'

In [49]:
# plusieurs maison 
c.loc[(c[(c['typ_lot']==1) & (c['Maison']>1)]).index, 'local'] = 'Lotissement'
c.loc[(c[(c['typ_lot']>1) & (c['Maison']>1) & (c['Appartement']==0) & (c['local']=='Maison')]).index, 'local'] = 'Lotissement'
c.loc[(c[(c['typ_lot']>1) & (c['Maison']>1) & (c['Appartement']!=0) & (c['local']=='Maison')]).index, 'local'] = 'mix'

In [50]:
# plusieurs local
c.loc[(c[(c['typ_lot']==1) & (c['Local industriel. commercial ou assimilé']>1)]).index, 'local'] = 'Centre commercial'
c.loc[(c[(c['typ_lot']>1) & (c['Local industriel. commercial ou assimilé']>1) & (c['Appartement']!=0) & (c['local']=='Local industriel. commercial ou assimilé')]).index, 'local'] = 'mix'
c.loc[(c[(c['typ_lot']>1) & (c['Local industriel. commercial ou assimilé']>1) & (c['Maison']!=0) & (c['local']=='Local industriel. commercial ou assimilé')]).index, 'local'] = 'mix'

In [51]:
# bien vendu seul 
c.loc[(c[c['local']=='Appartement']).index, 'local'] = 'Seul'
c.loc[(c[c['local']=='Maison']).index, 'local'] = 'Seul'
c.loc[(c[c['local']=='Local industriel. commercial ou assimilé']).index, 'local'] = 'Seul'

In [52]:
# drop les dépandance seul
c.drop(c[c['local']=='Dépendance'].index, inplace=True)

In [53]:
# prix m²
c.loc[(c[c['Surface_reelle_bati']!=0]).index, 'prix_m2'] = c['Valeur_fonciere']/c['Surface_reelle_bati']
c.loc[(c[c['Surface_reelle_bati']==0]).index, 'prix_m2'] = c['Valeur_fonciere']/c['Surface_terrain']

In [54]:
# drop bien avec des piece -9 m²
c.drop(c[(c['Nombre_pieces_principales']!=0) & ((c['Surface_reelle_bati']/c['Nombre_pieces_principales'])<9)  ].index, inplace=True)
# drop bien avec valeur nulle
c.drop(c[c['Valeur_fonciere']==0].index, inplace=True)
# drop bien avec prix m² moins que 400 e
c.drop(c[c['prix_m2'] < 400 ].index, inplace=True)
# drop bien avec prix m² superieur à 25000 e 
c.drop(c[c['prix_m2']>25000].index, inplace=True)

In [55]:
df.drop(columns=['tot_lot','typ_lot','Id'], inplace=True)

,Id,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison,Terrain,valeur,nbr_piece,tot_lot,typ_lot,surf_bati,surf terr,local,prix_m2
0,Vente 01/01/2020 13 207 E 195000.0,1,1,0,0,0,195000.0,2,2,2,53.0,0.0,Seul,3679.245283
1,Vente 01/01/2020 29 259 AM 87500.0,1,1,0,0,0,87500.0,3,2,2,62.0,0.0,Seul,1411.290323
3,Vente 01/01/2020 49 278 AM 100000.0,0,0,0,1,0,100000.0,3,1,1,75.0,122.0,Seul,1333.333333
7,Vente 01/02/2020 1 29 ZS 164000.0,0,0,0,1,2,164000.0,7,3,2,181.0,1771.0,Seul,906.077348
9,Vente 01/02/2020 1 53 AI 75000.0,1,0,0,0,0,75000.0,2,1,1,50.0,0.0,Seul,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028724,Vente 31/12/2020 974 18 AT 153450.0,0,0,0,0,1,153450.0,0,1,1,0.0,279.0,Terrain,550.000000
1028726,Vente 31/12/2020 974 7 AE 55500.0,1,1,0,0,0,55500.0,1,2,2,28.0,0.0,Seul,1982.142857
1028727,Vente 31/12/2020 974 8 AC 52650.0,0,0,1,0,1,52650.0,0,2,2,20.0,390.0,Seul,2632.500000
1028728,Vente 31/12/2020 974 8 AR 368000.0,0,0,0,1,0,368000.0,5,1,1,95.0,486.0,Seul,3873.684211


In [70]:
c[(c['local']=='Batiment')&c['Maison']!=0]

,Id,Appartement,Dépendance,Local industriel. commercial ou assimilé,Maison,Terrain,valeur,nbr_piece,tot_lot,typ_lot,surf_bati,surf terr,local,prix_m2


In [60]:
record(c,'df_final','CURATED')